## <font color='red'> INSTRUCTIONS </font>

<b> 
1. Write your code only in cells below the "WRITE CODE BELOW" title. Do not modify the code below the "DO NOT MODIFY" title. <br>
2. The expected data types of the output answers for each question are given in the last cell through assertion statements. Your answers must match these expected output data types. Hint: Many of the answers need to be a Python dictionary. Consider methods like to_dict() to convert a Pandas Series to a dictionary. <br>
3. The answers are then written to a JSON file named my_results_PA1.json. You can compare this with the provided expected output file "expected_results_PA1.json". <br>
4. After you complete writing your code, click "Kernel -> Restart Kernel and Run All Cells" on the top toolbar. There should NOT be any syntax/runtime errors, otherwise points will be deducted. <br>
5. For submitting your solution, first download your notebook by clicking "File -> Download". Rename the file as &ltTEAM_ID&gt.ipynb" and upload to Canvas.</b>


## <font color='red'> DO NOT MODIFY </font>

In [1]:
import time
import json
import dask
import dask.dataframe as dd
import pandas as pd
import ast
import re
from dask.distributed import Client
import ctypes
import numpy as np

def trim_memory() -> int:
    """
    helps to fix any memory leaks.
    """
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

client = Client("127.0.0.1:8786")
client.run(trim_memory)
client.restart()
print(client)

<Client: 'tcp://172.31.43.101:8786' processes=8 threads=8, memory=31.23 GiB>


In [2]:
start = time.time()

## <font color='blue'> WRITE CODE BELOW </font>

In [3]:
# Read user_reviews.csv with specified blocksize
user_reviews = dd.read_csv(
    'user_reviews.csv',
    blocksize='64MB', 
)

# Read products.csv with specified blocksize
products = dd.read_csv(
    'products.csv',
    blocksize='64MB',
    dtype={'asin': 'object'}
)

# Q1: Percentage of missing values in user_reviews
ans1 = (user_reviews.isnull().sum() / user_reviews.shape[0]).compute() * 100
ans1 = ans1.round(2).to_dict()

# Q2: Percentage of missing values in products
ans2 = (products.isnull().sum() / products.shape[0]).compute() * 100
ans2 = ans2.round(2).to_dict()


# Q3: Pearson correlation between price and overall rating
products['price'] = dd.to_numeric(products['price'], errors='coerce')
user_reviews['overall'] = dd.to_numeric(user_reviews['overall'], errors='coerce')

# Only selecting the columns needed for merging and correlation
merged = user_reviews[['asin', 'overall']].merge(
                    products[['asin', 'price']], on='asin', how='inner'
                    ).dropna(subset=['price', 'overall']).persist()

correlation_matrix = merged[['price', 'overall']].corr().compute()
ans3 = correlation_matrix.loc['price', 'overall'].round(2)
ans3 = float(ans3)

# Q4: Price statistics in products
price_clean = products['price'].persist()

# Compute summary stats
price_stats = price_clean.describe().compute()
# Convert to plain float

ans4 = {
    'mean': float(price_stats['mean']),
    'std': float(price_stats['std']),
    'min': float(price_stats['min']),
    'max': float(price_stats['max']),
    'median': float(price_stats['50%'])
}

# Q5: Number of products per super-category (sorted descending)
def extract_super_category(cat_string):
    try:
        categories_list = ast.literal_eval(cat_string)
        if isinstance(categories_list, list) and len(categories_list) > 0:
            return categories_list[0][0]
    except Exception:
        return None

products['super_category'] = products['categories'].map(extract_super_category, meta=('super_category', str))
super_category_counts = products['super_category'].dropna().value_counts().compute()

# Sort by counts descending
super_category_counts = super_category_counts.sort_values(ascending=False)

# Remove empty category keys
if '' in super_category_counts:
    super_category_counts = super_category_counts.drop('')

ans5 = super_category_counts.to_dict()

# Q6: Dangling reference check from product ids in "related" column to "asin" column of product table
review_asins = user_reviews[['asin']].dropna().drop_duplicates()
product_asins = products[['asin']].dropna().drop_duplicates().persist()

# Perform left merge to find dangling references
dangling = review_asins.merge(product_asins, on='asin', how='left', indicator=True)
dangling_only = dangling[dangling['_merge'] == 'left_only']

# Check if there are any dangling references
ans6 = int(dangling_only.shape[0].compute() > 0)

# Q7: Dangling reference check from product ids in "related" column to "asin" column of product table
def extract_related_asins(related):
    if isinstance(related, str):
        try:
            related = eval(related)  # evaluate string to dict
        except:
            return []
    if isinstance(related, dict):
        return [item for sublist in related.values() for item in sublist]
    return []

related_asins = (
    products['related']
    .dropna()
    .map(extract_related_asins, meta=('related', 'object'))
    .explode()
    .dropna()
    .to_frame(name='asin')
    .drop_duplicates()
    .persist()
)

dangling = related_asins.merge(product_asins, on='asin', how='left', indicator=True)
dangling_only = dangling[dangling['_merge'] == 'left_only']

ans7 = int(dangling_only.shape[0].compute() > 0)

/home/ubuntu/dask_env/lib/python3.10/site-packages/dask/dataframe/multi.py:169: UserWarning: Merging dataframes with merge column data type mismatches: 
+------------------+------------+-------------+
| Merge columns    | left dtype | right dtype |
+------------------+------------+-------------+
| ('asin', 'asin') | object     | string      |
+------------------+------------+-------------+
Cast dtypes explicitly to avoid unexpected results.
  warnings.warn(


## <font color='red'> DO NOT MODIFY </font>

In [4]:
end = time.time()

In [5]:
print(f"execution time = {end-start}s")

execution time = 298.9169328212738s


In [6]:
# DO NOT MODIFY
assert type(ans1) == dict, f"answer to question 1 must be a dictionary like {{'reviewerID':0.2, ..}}, got type = {type(ans1)}"
assert type(ans2) == dict, f"answer to question 2 must be a dictionary like {{'asin':0.2, ..}}, got type = {type(ans2)}"
assert type(ans3) == float, f"answer to question 3 must be a float like 0.8, got type = {type(ans3)}"
assert type(ans4) == dict, f"answer to question 4 must be a dictionary like {{'mean':0.4,'max':0.6,'median':0.6...}}, got type = {type(ans4)}"
assert type(ans5) == dict, f"answer to question 5 must be a dictionary, got type = {type(ans5)}"         
assert ans6 == 0 or ans6==1, f"answer to question 6 must be 0 or 1, got value = {ans6}" 
assert ans7 == 0 or ans7==1, f"answer to question 7 must be 0 or 1, got value = {ans7}" 

ans_dict = {
    "q1": ans1,
    "q2": ans2,
    "q3": ans3,
    "q4": ans4,
    "q5": ans5,
    "q6": ans6,
    "q7": ans7,
    "runtime": end-start
}
with open('my_results_PA1.json', 'w') as outfile: json.dump(ans_dict, outfile)         